### Load reqs and start

In [1]:
import sys, os
sys.path.append("../src/python") 

In [2]:
cd "/home/iceberg"


/home/iceberg


/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
pip install -e /home/iceberg

Obtaining file:///home/iceberg
  Installing build dependencies ... one
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... one
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for customer_transaction_etl (pyproject.toml) ... done
  Created wheel for customer_transaction_etl: filename=customer_transaction_etl-0.1.0-0.editable-py3-none-any.whl size=1441 sha256=65c27dcae07bd9abb8fe071d05821190427c6651a6227da5173a3f9b6d754929
  Stored in directory: /tmp/pip-ephem-wheel-cache-61bscaoi/wheels/47/e4/18/3729c1b70e0dfec9a6b23d35b4183f467ede935f2336901bfb
Successfully built customer_transaction_etl
  Attempting uninstall: customer_transaction_etl
    Found existing installation: customer_transaction_etl 0.1.0
    Uninstalling customer_transaction_etl-0.1.0:
      Successfully uninstalled customer_transaction_etl-0.1.0

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install -

## Starting the Data Exploration 

In [4]:
from customer_transaction_etl.session import build_spark
spark = build_spark("data-exploration")

25/09/28 10:53:48 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [40]:
from datetime import datetime
print(datetime.today().strftime('%d-%m-%Y'))
today=datetime.today().strftime('%d-%m-%Y')

from customer_transaction_etl.config import Paths
ingest_date = "2025-09-25"  # or today 
source_root = "/home/iceberg/data/raw/customer_data_ingest"  # <- adjust if needed
raw_glob = f"{source_root}/ingest_date={ingest_date}/*.json"
#raw_glob = os.path.join(paths.raw, f"ingest_date={ingest_date}", "*.json")
print("Reading:", raw_glob)

28-09-2025
Reading: /home/iceberg/data/raw/customer_data_ingest/ingest_date=2025-09-25/*.json


In [44]:
print("Reading:", raw_glob)
df_raw = spark.read.json(raw_glob, multiLine=True)

df_raw.printSchema()
df_raw.show(truncate=False)

Reading: /home/iceberg/data/raw/customer_data_ingest/ingest_date=2025-09-25/*.json
root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_id: string (nullable = true)
 |    |    |-- quantity: string (nullable = true)
 |-- purchase_date: string (nullable = true)

+-----------+--------+---------------------------------------------------------------+-------------+
|customer_id|order_id|products                                                       |purchase_date|
+-----------+--------+---------------------------------------------------------------+-------------+
|C001       |O001    |[{Product A, 10,99, P001, 2,00}, { Product B,  5.99 , P002, 1}]| 2023-01-01  |
|C002       |O002    |[{Product A, 10,99, P001, 3,00}, {Product C, 7.99, P003,  2}]  |2023-01-02 

## Data Transformation

### <span style="color:blue"> ExplodeToLines </span>

In [45]:
from customer_transaction_etl.steps import ExplodeToLines

explode = ExplodeToLines()

df_lines = explode.transform(df_raw)

df_lines.printSchema()
df_lines.show(truncate=False)


root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- purchase_date: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- price: string (nullable = true)

+--------+-----------+-------------+----------+------------+--------+------+
|order_id|customer_id|purchase_date|product_id|product_name|quantity|price |
+--------+-----------+-------------+----------+------------+--------+------+
|O001    |C001       | 2023-01-01  |P001      |Product A   |2,00    |10,99 |
|O001    |C001       | 2023-01-01  |P002      | Product B  |1       | 5.99 |
|O002    |C002       |2023-01-02   |P001      |Product A   |3,00    |10,99 |
|O002    |C002       |2023-01-02   |P003      |Product C   | 2      |7.99  |
|O003    |C003       |2023-01-03   |P002      |Product B   | 1      |5,99  |
|O003    |C003       |2023-01-03   |P003      |Product C   |4,00    | 7,99 |
+--------+

#### How it will look without calling the class

In [52]:

from pyspark.sql import DataFrame, functions as F
exploded_exmp = df_raw.withColumn("item", F.explode("products"))

exploded_exmp.printSchema()
exploded_exmp.show()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_id: string (nullable = true)
 |    |    |-- quantity: string (nullable = true)
 |-- purchase_date: string (nullable = true)
 |-- item: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- product_id: string (nullable = true)
 |    |-- quantity: string (nullable = true)

+-----------+--------+--------------------+-------------+--------------------+
|customer_id|order_id|            products|purchase_date|                item|
+-----------+--------+--------------------+-------------+--------------------+
|       C001|    O001|[{Product A, 10,9...|   2023-01-01|{Product A, 10,99...|
|       C001|    O001|[{Product A, 10,9...|   2023-01-01|

In [54]:
        lines_exmp = (
            exploded_exmp
            .select(
                F.col("order_id"),
                F.col("customer_id"),
                F.col("purchase_date"),
                F.col("item.product_id").alias("product_id"),
                F.col("item.name").alias("product_name"),
                F.col("item.quantity").alias("quantity"),
                F.col("item.price").alias("price"),
            )
        )   

lines_exmp.printSchema()
lines_exmp.show()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- purchase_date: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- price: string (nullable = true)

+--------+-----------+-------------+----------+------------+--------+------+
|order_id|customer_id|purchase_date|product_id|product_name|quantity| price|
+--------+-----------+-------------+----------+------------+--------+------+
|    O001|       C001|   2023-01-01|      P001|   Product A|    2,00| 10,99|
|    O001|       C001|   2023-01-01|      P002|   Product B|       1| 5.99 |
|    O002|       C002|  2023-01-02 |      P001|   Product A|    3,00| 10,99|
|    O002|       C002|  2023-01-02 |      P003|   Product C|       2|  7.99|
|    O003|       C003|   2023-01-03|      P002|   Product B|       1|  5,99|
|    O003|       C003|   2023-01-03|      P003|   Product C|    4,00| 7,99 |
+--------+

#### End Example

#### Sanity Checks

In [57]:

# 1) Number of line rows should equal sum of products array sizes per order
expected_count = (df_raw
    .select(F.size(F.col("products")).alias("n"))
    .agg(F.sum("n").alias("sum_n"))
    .collect()[0]["sum_n"])

actual_count = df_lines.count()

print("Expected line count:", expected_count, " | Actual:", actual_count)

Expected line count: 6  | Actual: 6


In [59]:
# 2) No columns lost accidentally
print("Columns:", df_lines.columns)

# 3) Quick peek grouped by order to ensure row multiplication looks right
df_lines.groupBy("order_id").agg(F.count("*").alias("line_count")).show()

Columns: ['order_id', 'customer_id', 'purchase_date', 'product_id', 'product_name', 'quantity', 'price']
+--------+----------+
|order_id|line_count|
+--------+----------+
|    O002|         2|
|    O001|         2|
|    O003|         2|
+--------+----------+



### <span style="color:blue"> CleanseAndCast</span>

In [79]:
from customer_transaction_etl.steps import CleanseAndCast

In [80]:
cast_step = CleanseAndCast()
df_cast = cast_step.transform(df_lines)  # df_lines is the output from ExplodeToLines

df_cast.printSchema()
df_cast.show(10, truncate=False)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- purchase_date: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: decimal(10,2) (nullable = true)

+--------+-----------+-------------+----------+------------+--------+-----+
|order_id|customer_id|purchase_date|product_id|product_name|quantity|price|
+--------+-----------+-------------+----------+------------+--------+-----+
|O001    |C001       | 2023-01-01  |P001      |Product A   |NULL    |10.99|
|O001    |C001       | 2023-01-01  |P002      |Product B   |1       |5.99 |
|O002    |C002       |2023-01-02   |P001      |Product A   |NULL    |10.99|
|O002    |C002       |2023-01-02   |P003      |Product C   |2       |7.99 |
|O003    |C003       |2023-01-03   |P002      |Product B   |1       |5.99 |
|O003    |C003       |2023-01-03   |P003      |Product C   |NULL    |7.99 |
+--------+-

In [64]:
total_rows = df_cast.count()
print("Rows after cast:", total_rows)

# 1) How many rows failed to parse critical fields (now null)?
invalid = df_cast.where(
    F.col("order_id").isNull() |
    F.col("product_id").isNull() |
    F.col("purchase_date").isNull() |
    F.col("quantity").isNull() |
    F.col("price").isNull()
)
print("Invalid rows:", invalid.count())
invalid.show(truncate=False)

# 2) Are any quantities non-positive? (optional business rule)
neg_qty = df_cast.where(F.col("quantity") <= 0)
print("Non-positive quantities:", neg_qty.count())

# 3) Are any prices negative?
neg_price = df_cast.where(F.col("price") <= 0)
print("Non-positive prices:", neg_price.count())

Rows after cast: 6
Invalid rows: 3
+--------+-----------+-------------+----------+------------+--------+-----+
|order_id|customer_id|purchase_date|product_id|product_name|quantity|price|
+--------+-----------+-------------+----------+------------+--------+-----+
|O001    |C001       | 2023-01-01  |P001      |Product A   |NULL    |10.99|
|O002    |C002       |2023-01-02   |P001      |Product A   |NULL    |10.99|
|O003    |C003       |2023-01-03   |P003      |Product C   |NULL    |7.99 |
+--------+-----------+-------------+----------+------------+--------+-----+

Non-positive quantities: 0
Non-positive prices: 0


In [65]:
df_lines.select("order_id","product_id","quantity").show(truncate=False)

+--------+----------+--------+
|order_id|product_id|quantity|
+--------+----------+--------+
|O001    |P001      |2,00    |
|O001    |P002      |1       |
|O002    |P001      |3,00    |
|O002    |P003      | 2      |
|O003    |P002      | 1      |
|O003    |P003      |4,00    |
+--------+----------+--------+



In [66]:
from pyspark.sql import functions as F

df_lines.select(
    "order_id","product_id",
    F.col("quantity").alias("q_raw"),
    F.col("price").alias("p_raw"),
    F.col("purchase_date").alias("d_raw")
).show(truncate=False)

+--------+----------+-----+------+-----------+
|order_id|product_id|q_raw|p_raw |d_raw      |
+--------+----------+-----+------+-----------+
|O001    |P001      |2,00 |10,99 | 2023-01-01|
|O001    |P002      |1    | 5.99 | 2023-01-01|
|O002    |P001      |3,00 |10,99 |2023-01-02 |
|O002    |P003      | 2   |7.99  |2023-01-02 |
|O003    |P002      | 1   |5,99  |2023-01-03 |
|O003    |P003      |4,00 | 7,99 |2023-01-03 |
+--------+----------+-----+------+-----------+



In [75]:
norm = (df_lines
    .withColumn("q_trim", F.trim("quantity"))
    .withColumn("q_norm", F.regexp_replace(F.regexp_replace(F.col("q_trim"), r"\s+", ""), ",", "."))
    .withColumn("q_dbl",  F.col("q_norm").cast("double"))
    .withColumn("p_trim", F.trim("price"))
    .withColumn("p_norm", F.regexp_replace(F.regexp_replace(F.col("p_trim"), r"\s+", ""), ",", "."))
    .withColumn("p_dec",  F.col("p_norm").cast("decimal(10,2)"))
    .withColumn("d_trim", F.trim("purchase_date"))
    .withColumn("d1",     F.to_date("d_trim", "yyyy-MM-dd"))
    .withColumn("d2",     F.to_date("d_trim", "dd/MM/yyyy"))
    .withColumn("d_final",F.coalesce("d1","d2"))
)
norm.select("*").show(truncate=False)



+--------+-----------+-------------+----------+------------+--------+------+------+------+-----+------+------+-----+----------+----------+----+----------+
|order_id|customer_id|purchase_date|product_id|product_name|quantity|price |q_trim|q_norm|q_dbl|p_trim|p_norm|p_dec|d_trim    |d1        |d2  |d_final   |
+--------+-----------+-------------+----------+------------+--------+------+------+------+-----+------+------+-----+----------+----------+----+----------+
|O001    |C001       | 2023-01-01  |P001      |Product A   |2,00    |10,99 |2,00  |2.00  |2.0  |10,99 |10.99 |10.99|2023-01-01|2023-01-01|NULL|2023-01-01|
|O001    |C001       | 2023-01-01  |P002      | Product B  |1       | 5.99 |1     |1     |1.0  |5.99  |5.99  |5.99 |2023-01-01|2023-01-01|NULL|2023-01-01|
|O002    |C002       |2023-01-02   |P001      |Product A   |3,00    |10,99 |3,00  |3.00  |3.0  |10,99 |10.99 |10.99|2023-01-02|2023-01-02|NULL|2023-01-02|
|O002    |C002       |2023-01-02   |P003      |Product C   | 2      |7

In [70]:
df_lines.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- purchase_date: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- price: string (nullable = true)

